# Point Cloud AutoEncoders "Experimental"


In [1]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.in_out import create_dir

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
DATA_PATH = '/Users/optas/DATA/Point_Clouds/Shape_Net_Core/3000/no_segmentations/03001627/'
LOG_PATH = '/Users/optas/DATA/Neural_Nets/Train_Log/Point_Cloud_AE'
MODEL_PATH = '/Users/optas/DATA/Neural_Nets/Models/Point_Cloud_AE'
create_dir(LOG_PATH)
create_dir(MODEL_PATH)

In [4]:
QUEUE_EPOCH_FRACTION = 0.1
NUM_EPOCHS_PER_DECAY = 8          # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.001     # Initial learning rate.

#     keep_prob = 0.5
#     stddev = 5e-2

In [12]:
from tf_lab.point_clouds import point_cloud_ae as pae
from tf_lab.point_clouds.point_cloud_ae import Configuration as ae_conf
import tf_lab.point_clouds.in_out as pio
from tf_lab.fundamentals.loss import Loss


In [14]:
config = ae_conf()
in_signal, gt_signal = pio.in_out_placeholders(config)
model = pae.autoencoder_with_fcs_only(in_signal, config)

In [15]:
in_files = pio.load_filenames_of_input_data(DATA_PATH)

capacity = 10000
input_tensors = in_files

data_q = tf.RandomShuffleQueue(capacity=capacity, min_after_dequeue = capacity/4, dtypes=input_tensors.dtype)

6778 files containing  point clouds were found.


In [16]:

num_epochs = 10
epoch_tensors = []
for tensor in input_tensors:
    epoch_tensors.append(tf.train.limit_epochs(tensor, num_epochs))

print epoch_tensors
# enq_op = data_q.enqueue_many(epoch_tensors)
# qr = tf.train.QueueRunner(self.data_q, [enq_op])
#         tf.train.add_queue_runner(qr)
    

TypeError: 'Tensor' object is not iterable.

In [15]:
loss = Loss.l2_loss(model, gt_signal)
optimizer = tf.train.GradientDescentOptimizer(INITIAL_LEARNING_RATE).minimize(loss)
pclouds, names = load_crude_point_clouds(DATA_PATH)

In [20]:
total_steps = (len(pclouds) // config.batch_size) * config.training_epochs

In [15]:
with tf.Graph().as_default() as g:
    for step in total_steps:
    
        
        
#     # Get images and labels for CIFAR-10.
#     images, labels = cifar10.distorted_inputs()
#     pred = cifar_10_vgg_classifier(images)
#     #     fc_cv, fc = test_vgg_m_conv(images, 1, g)
#     init = tf.initialize_all_variables()        
#     sess = tf.Session(config=tf.ConfigProto(log_device_placement=False))
#     sess.run(init)
#     # Start the queue runners.
#     tf.train.start_queue_runners(sess=sess)    
#     print sess.run([pred])
    

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
[array([[  9.89372240e-29,   9.99751389e-01,   4.25720461e-35, ...,
          2.48523487e-04,   7.65052235e-08,   6.46529433e-19],
       [  5.58076707e-22,   3.49713904e-23,   1.40103731e-35, ...,
          9.08389919e-10,   1.72844185e-26,   7.37164357e-11],
       [  9.71022601e-27,   2.51064481e-23,   0.00000000e+00, ...,
          1.55498248e-36,   6.02137003e-15,   7.03709175e-37],
       ..., 
       [  5.50711560e-12,   2.81982924e-19,   0.00000000e+00, ...,
          4.71312746e-12,   1.00000000e+00,   1.39827672e-09],
       [  9.99974370e-01,   1.99023309e-09,   4.99290294e-16, ...,
          2.56190851e-05,   1.22772589e-14,   1.48472119e-14],
       [  1.66505838e-24,   7.60911569e-29,   1.34719100e-33, ...,
          2.27212929e-14,   1.00000000e+00,   5.11814809e-24]], dtype=float32)]


[-4.82317972  7.56947613  2.45779705 -1.21228302  5.42072678 -5.22100878
 -1.47143781  5.10664082  4.20103455 -0.52912915 -3.39429331  1.91564465
  1.12573886 -1.36975384 -2.57543731  3.38702774 -1.90229058 -7.67551136
  0.73936599 -1.94913518  3.53006053  0.15869325  5.00703335 -0.71717119
  4.98837757  0.75052369  1.19316506 -1.38610315 -2.14804769 -0.78406906]
[-4.82317972  7.56947613  2.45779705 -1.21228302  5.42072678 -5.22100878
 -1.47143781  5.10664082  4.20103455 -0.52912915 -3.39429331  1.91564465
  1.12573886 -1.36975384 -2.57543731  3.38702774 -1.90229058 -7.67551136
  0.73936599 -1.94913518  3.53006053  0.15869325  5.00703335 -0.71717119
  4.98837757  0.75052369  1.19316506 -1.38610315 -2.14804769 -0.78406906]
